<img width="8%" alt="Trello.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Trello.png" style="border-radius: 15%">

# Trello - Get Cards on a Board
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Trello+-+Get+Cards+on+a+Board:+Error+short+description">Bug report</a>

**Tags:** #trello #api #rest #cards #board #get

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Last update:** 2023-07-14 (Created: 2023-07-14)

**Description:** This notebook get all cards from a board.

**References:**
- [Trello API Documentation](https://developer.atlassian.com/cloud/trello/rest/api-group-boards/#api-boards-id-cards-get)
- [Trello API Authentication](https://developer.atlassian.com/cloud/trello/guides/rest-api/api-introduction/#authentication)
-  [Get your Trello API key](https://github.com/Benjifilly/My_notebooks/wiki/How-to-obtain-an-API-key-for-Trello)

## Input

### Import libraries

In [ ]:
import requests
import naas

### Setup variables
- `api_key`: Your Trello API key.
- `force_update`: Parameters to enforce the token update
- `board_id`: The ID of the board you want to get the cards from

In [ ]:
api_key = naas.secret.get("TRELLO_API_KEY") or "YOUR_TRELLO_API_KEY"
force_update = False
board_id = '64a19089d2xxxxxx' #Example: "https://trello.com/b/ogBhOe9a/ultimate-to-do-list"

## Model

### Get your token
Your Trello token will be stored on naas secret name: "TRELLO_TOKEN"

In [ ]:
def get_token(api_key, force_update):
    token = naas.secret.get("TRELLO_TOKEN")
    if not token or force_update:
        url = f"https://trello.com/1/authorize?expiration=never&scope=read,write,account&response_type=token&key={api_key}"
        print(f"🔗 Click on the URL to get your token: {url}")
        print("✅ On the authorization page, click on 'Authorize' button at the bottom of the page to proceed.")
        print("💾 Copy/Paste the token in red in the input case below")
        token = input("-> Enter your token here:")
        naas.secret.add("TRELLO_TOKEN", token)
    return token
     
token = get_token(api_key, force_update)

### Get Cards on a Board

This function will get all of the open Cards on a Board.

In [ ]:
# Endpoint URL to get all lists on a board
lists_url = f'https://api.trello.com/1/boards/{board_id}/lists'

# Parameters for the lists request
lists_params = {
    'key': api_key,
    'token': token,
}

# Send GET request to the Trello API to get lists
lists_response = requests.get(lists_url, params=lists_params)

## Output

### Display result

In [ ]:
# Check if the lists request was successful
if lists_response.status_code == 200:
    # Parse the lists response as JSON
    lists = lists_response.json()

    # Iterate through the lists
    for list_obj in lists:
        list_id = list_obj['id']

        # Endpoint URL to get cards in a list
        cards_url = f'https://api.trello.com/1/lists/{list_id}/cards'

        # Parameters for the cards request
        cards_params = {
            'key': api_key,
            'token': token,
        }

        # Send GET request to the Trello API to get cards
        cards_response = requests.get(cards_url, params=cards_params)

        # Check if the cards request was successful
        if cards_response.status_code == 200:
            # Parse the cards response as JSON
            cards = cards_response.json()
            cards_count = len(cards)

            # Print the list title with count of cards
            print(f"=== Card List ===")
            print(f"{list_obj['name']} : {cards_count}")

            # Print the card names
            for card in cards:
                print(f"📌 {card['name']}")

            # Print a separator line between lists
            print("-----------------------------------------")

        else:
            print(f"Error: {cards_response.status_code} - {cards_response.text}")

else:
    print(f"Error: {lists_response.status_code} - {lists_response.text}")